# Imports

In [ ]:
!pip install scikit-learn-extra
!pip install lime

In [ ]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import string
import re
import argparse
import json
from sklearn import neighbors, datasets

import os
import sys
import random
import shutil
import time
import torch.nn.parallel
from tqdm import tqdm
import torch.optim
import torch.utils.data
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import multivariate_normal
import  scipy.stats as st
from matplotlib import cm
from __future__ import print_function
from spacy.lang.en import English
from sklearn.cluster import KMeans
import pickle
import matplotlib
from sklearn.metrics.pairwise import rbf_kernel
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
import scipy.stats as st
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lime
import lime.lime_tabular


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(66)
print(device)

# CIFAR Model

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                                                                padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)


class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)


class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
        assert ((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]
        self.softmax = nn.Softmax()
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        out = self.fc(out)
        out = self.softmax(out)
        return out
    def get_repr(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return out

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def metrics_print(net,expert_fn, n_classes, loader):
    '''
    Computes metrics for deferal
    -----
    Arguments:
    net: model
    expert_fn: expert model
    n_classes: number of classes
    loader: data loader
    '''
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    alone_correct = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            batch_size = outputs.size()[0]  # batch_size
            exp_prediction = expert_fn(images, labels)
            for i in range(0, batch_size):
                r = (predicted[i].item() == n_classes)
                prediction = predicted[i]
                if predicted[i] == n_classes:
                    max_idx = 0
                    # get second max
                    for j in range(0, n_classes):
                        if outputs.data[i][j] >= outputs.data[i][max_idx]:
                            max_idx = j
                    prediction = max_idx
                else:
                    prediction = predicted[i]
                alone_correct += (prediction == labels[i]).item()
                if r == 0:
                    total += 1
                    correct += (predicted[i] == labels[i]).item()
                    correct_sys += (predicted[i] == labels[i]).item()
                if r == 1:
                    exp += (exp_prediction[i] == labels[i].item())
                    correct_sys += (exp_prediction[i] == labels[i].item())
                    exp_total += 1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print = {"coverage": cov, "system accuracy": 100 * correct_sys / real_total,
                "expert accuracy": 100 * exp / (exp_total + 0.0002),
                "classifier accuracy": 100 * correct / (total + 0.0001),
                "alone classifier": 100 * alone_correct / real_total}
    print(to_print)

def metrics_print_baseline(net_class, expert_fn, n_classes, loader):
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs_class = net_class(images)
            _, predicted = torch.max(outputs_class.data, 1)
            batch_size = outputs_class.size()[0]  # batch_size

            exp_prediction = expert_fn(images, labels)
            for i in range(0, batch_size):
                r = (exp_prediction[i] == labels[i].item())
                if r == 0:
                    total += 1
                    prediction = predicted[i]
                    if predicted[i] == n_classes:
                        max_idx = 0
                        for j in range(0, n_classes):
                            if outputs_class.data[i][j] >= outputs_class.data[i][max_idx]:
                                max_idx = j
                        prediction = max_idx
                    else:
                        prediction = predicted[i]
                    correct += (prediction == labels[i]).item()
                    correct_sys += (prediction == labels[i]).item()
                if r == 1:
                    exp += (exp_prediction[i] == labels[i].item())
                    correct_sys += (exp_prediction[i] == labels[i].item())
                    exp_total += 1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print = {"coverage": cov, "system accuracy": 100 * correct_sys / real_total,
                "expert accuracy": 100 * exp / (exp_total + 0.0002),
                "classifier accuracy": 100 * correct / (total + 0.0001)}
    print(to_print)

In [ ]:
def my_CrossEntropyLoss(outputs, labels):
    batch_size = outputs.size()[0]  # batch_size
    outputs = - torch.log2(outputs[range(batch_size), labels])  # pick the values corresponding to the labels
    return torch.sum(outputs) / batch_size

def train_classifier(train_loader, model, optimizer, scheduler, epoch, expert_fn, n_classes):
    """Train for one epoch on the training set"""
    # expertfn: a number here k 
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        target = target.to(device)
        input = input.to(device)

        # compute output
        output = model(input)
        # compute loss
        loss = my_CrossEntropyLoss(output, target)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                loss=losses, top1=top1))


def validate_classifier(val_loader, model, epoch, expert_fn, n_classes):
    """Perform validation on the validation set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.to(device)
        input = input.to(device)

        # compute output
        with torch.no_grad():
            output = model(input)
        # compute loss
        loss = my_CrossEntropyLoss(output, target)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                i, len(val_loader), batch_time=batch_time, loss=losses,
                top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg
best_prec1 = 0
def run_classifier(model, data_aug, n_dataset, expert_fn, epochs):
    global best_prec1
    

    # get the number of model parameters
    print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()])))

    # for training on multiple GPUs.
    # Use CUDA_VISIBLE_DEVICES=0,1 to specify which GPUs to use
    # model = torch.nn.DataParallel(model).cuda()
    model = model.to(device)

    # optionally resume from a checkpoint

    cudnn.benchmark = True

    # define loss function (criterion) and optimizer
    optimizer = torch.optim.SGD(model.parameters(), 0.1,
                                momentum=0.9, nesterov=True,
                                weight_decay=5e-4)

    # cosine learning rate
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * 200)

    for epoch in range(0, epochs):
        # train for one epoch
        if epoch % 10 ==0:
            validate_classifier(train_val_loader, model_classifier, None, None, 10)
        train_classifier(train_loader, model, optimizer, scheduler, epoch, expert_fn, n_dataset)

In [ ]:

data_aug = False
n_dataset = 10
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

if data_aug:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: F.pad(x.unsqueeze(0),
                                            (4, 4, 4, 4), mode='reflect').squeeze()),
        transforms.ToPILImage(),
        transforms.RandomCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

if n_dataset == 10:
    dataset = 'cifar10'
elif n_dataset == 100:
    dataset = 'cifar100'

kwargs = {'num_workers': 0, 'pin_memory': True}




train_dataset_all = datasets.__dict__[dataset.upper()]('../data', train=True, download=True,
                                                        transform=transform_train)
train_size = int(0.90 * len(train_dataset_all))
test_size = len(train_dataset_all) - train_size

train_dataset, train_test_dataset = torch.utils.data.random_split(train_dataset_all, [train_size, test_size],  generator=torch.Generator().manual_seed(66))
train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=128, shuffle=True, **kwargs)
train_val_loader = torch.utils.data.DataLoader(train_test_dataset,
                                            batch_size=128, shuffle=False, **kwargs)

test_dataset_all = datasets.__dict__[dataset.upper()]('../data', train=False, download=True,
                                                        transform=transform_train)
train_size = int(0.5 * len(train_dataset_all))
test_size = len(train_dataset_all) - train_size

test_dataset, val_dataset = torch.utils.data.random_split(train_dataset_all, [train_size, test_size], generator=torch.Generator().manual_seed(66))

val_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=128, shuffle=False, **kwargs)

test_all_loader = torch.utils.data.DataLoader(test_dataset_all,
                                            batch_size=128, shuffle=False, **kwargs)


test_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=128, shuffle=False, **kwargs)


# Loading model Code

In [ ]:
n_dataset = 10  # cifar-10, 100 for cifar-100
model_classifier = WideResNet(28, n_dataset, 4, dropRate=0)
model_classifier = torch.load("cifar_model_seed66_5k.pt").to(device)

# Training model code

In [ ]:
n_dataset = 10  # cifar-10, 100 for cifar-100
model_classifier = WideResNet(28, n_dataset, 4, dropRate=0)

In [ ]:
run_classifier(model_classifier, False, n_dataset, 0, 200)
#model_classifier = torch.load("model_classifier_cifar_seed66.pt").to(device)

In [ ]:
model_classifier = torch.load("cifar_model_seed66.pt").to(device)

In [ ]:
torch.save(model_classifier, "cifar_model_seed66_5k.pt")

In [ ]:
model_classifier = torch.load("cifar_model_seed66_5k.pt").to(device)

In [ ]:
validate_classifier(test_all_loader, model_classifier, None, None, 10)


# Classes for Teaching

In [ ]:
class HumanLearner:
    def __init__(self, kernel):
        '''
        kernel: function that takes two inputs and returns a similarity
        prior rejector: returns rejector
        '''
        self.teaching_set = []
        self.kernel = kernel
        self.rejector_tresh = 0.8

    def predict(self, xs, prior_rejector_preds, to_print = False):
        '''
        xs: expected array of inputs
        '''
        preds = []
        idx = 0
        used_posterior = 0 
        if to_print:
            print("-- Human making reject predictions --")
            with tqdm(total=len(xs)) as pbar:
                for x in xs:
                    ball_at_x = []
                    similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                    for i in range(len(self.teaching_set)):
                        similarity = similarities[i]
                        if similarity >=  self.teaching_set[i][2]:
                            ball_at_x.append(self.teaching_set[i])
                    if len(ball_at_x) == 0: 
                        # use prior rejector
                        preds.append(prior_rejector_preds[idx])
                    else:
                        used_posterior += 1
                        ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0] for kk in range(len(ball_at_x))]))[0]
                        normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                        score_one = np.sum([ball_similarities[i]*ball_at_x[i][1] for i in range(len(ball_at_x))])
                        pred = score_one / normalization
                        if pred >= 0.5:
                            preds.append(1)
                        else:
                            preds.append(0)
                    idx += 1
                    pbar.update(1)
        else:
            for x in xs:
                ball_at_x = []
                similarities = rbf_kernel(x.reshape(1,-1), np.asarray([self.teaching_set[kk][0] for kk in range(len(self.teaching_set))]))[0]
                for i in range(len(self.teaching_set)):
                    similarity = similarities[i]
                    if similarity >=  self.teaching_set[i][2]:
                        ball_at_x.append(self.teaching_set[i])
                if len(ball_at_x) == 0: 
                    # use prior rejector
                    preds.append(prior_rejector_preds[idx])
                else:
                    used_posterior += 1
                    ball_similarities = rbf_kernel(x.reshape(1,-1), np.asarray([ball_at_x[kk][0] for kk in range(len(ball_at_x))]))[0]
                    normalization = np.sum([ball_similarities[i] for i in range(len(ball_at_x))])
                    score_one = np.sum([ball_similarities[i]*ball_at_x[i][1] for i in range(len(ball_at_x))])
                    pred = score_one / normalization
                    if pred >= 0.5:
                        preds.append(1)
                    else:
                        preds.append(0)
                idx += 1
        if to_print:
            print(f'Used posterior {used_posterior/len(xs)*100:.2f}')
        return preds

    def add_to_teaching(self, teaching_example):
        '''
        teaching_example: (x, label, gamma)
        '''
        self.teaching_set.append(teaching_example)

    def remove_last_teaching_item(self):
        self.teaching_set = self.teaching_set[:-1]

    def prior_rejector(self, ai_confidence = 0):
        coin = random.random() # random number between [0,1]
        if coin >= 0.5:
            return 1
        return 0


In [ ]:
def compute_predictions_humanai(hum_preds, hum_rejector, ai_preds, data_x):
    '''
    hum_preds: array of human predictions
    ai_preds: array of AI predictions
    hum_rejector: HumanLearner
    data_x: array of inputs

    Returns array of final predictions and deferalls
    '''
    predictions = []
    with torch.no_grad():
        reject_decisions = hum_rejector(data_x)
        for i in range(len(data_x)):
            if reject_decisions[i] == 1:
                # defer
                predictions.append(ai_preds[i])
            else:
                predictions.append(hum_preds[i])
    return predictions, reject_decisions

def get_metrics(preds, truths):
    '''
    preds: array of predictions
    truths:  target array
    '''
    acc = metrics.accuracy_score(truths, preds)
    metrics_computed = { "accuracy": acc}
    return metrics_computed

def compute_metrics(human_preds, ai_preds, reject_decisions, truths, to_print = False):
    coverage = 1 - np.sum(reject_decisions)/len(reject_decisions)
    humanai_preds = []
    human_preds_sys = []
    truths_human = []
    ai_preds_sys = []
    truths_ai = []
    for i in range(len(reject_decisions)):
        if reject_decisions[i] == 1:
            humanai_preds.append(ai_preds[i])
            ai_preds_sys.append(ai_preds[i])
            truths_ai.append(truths[i])
        else:
            humanai_preds.append(human_preds[i])
            human_preds_sys.append(human_preds[i])
            truths_human.append(truths[i])
    humanai_metrics = get_metrics(humanai_preds, truths)

    human_metrics = get_metrics(human_preds_sys, truths_human)

    ai_metrics = get_metrics(ai_preds_sys, truths_ai)

    if to_print:
        print(f'Coverage is {coverage*100:.2f}')
        print(f' metrics of system are: {humanai_metrics}')
        print(f' metrics of human are: {human_metrics}')
        print(f' metrics of AI are: {ai_metrics}')
    return coverage, humanai_metrics, human_metrics, ai_metrics

In [ ]:
class synth_expert:
    '''
    From https://github.com/clinicalml/learn-to-defer
    expert who is perfect if class is in [0,k] and otherwise is uniform at random
    k: [0,n_classes] 
    n_classes: number of classes in data
    '''
    def __init__(self, k, n_classes):
        self.k = k
        self.n_classes = n_classes

    def predict(self, input, labels):
        batch_size = labels.size()[0]  # batch_size
        outs = [0] * batch_size
        for i in range(0, batch_size):
            if labels[i].item() <= self.k:
                outs[i] = labels[i].item()
            else:
                prediction_rand = random.randint(0, self.n_classes - 1)
                outs[i] = prediction_rand
        return outs

class synth_expert_random:
    '''
    expert who is accurate at random with probability ~acc_level 
    acc_level: [0,1] probability of being correct
    n_classes: number of classes in data
    '''
    def __init__(self, acc_level, n_classes):
        self.acc_level = acc_level
        self.n_classes = n_classes

    def predict(self, input, labels):
        batch_size = labels.size()[0]  # batch_size
        outs = [0] * batch_size
        for i in range(0, batch_size):
            coin = random.random()
            if coin <= self.acc_level:
                outs[i] = labels[i].item()
            else:
                prediction_rand = random.randint(0, self.n_classes - 1)
                outs[i] = prediction_rand
        return outs


expert = synth_expert_random(0.7, 10)

In [ ]:
def metrics_print_baseline(net_class, expert_fn, n_classes, loader, epsilon):
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs_class = net_class(images)
            _, predicted = torch.max(outputs_class.data, 1)
            batch_size = outputs_class.size()[0]  # batch_size
            exp_prediction = expert_fn(images, labels)
            for i in range(0, batch_size):
                r = (max(outputs_class[i]) <= epsilon )
                if r == 0:
                    total += 1
                    correct += (predicted[i] == labels[i]).item()
                    correct_sys += (predicted[i] == labels[i]).item()
                if r == 1:
                    exp += (exp_prediction[i] == labels[i].item())
                    correct_sys += (exp_prediction[i] == labels[i].item())
                    exp_total += 1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print = {"coverage": cov, "system accuracy": 100 * correct_sys / real_total,
                "expert accuracy": 100 * exp / (exp_total + 0.0002),
                "classifier accuracy": 100 * correct / (total + 0.0001)}
    #print(to_print)
    return to_print

In [ ]:
# get optimal gammas, ONLY FOR TEACHING
def get_optimal_gammas():
    optimal_gammas = []
    with tqdm(total=len(teaching_embeddings)) as pbar:
        similarities_embeds_all = rbf_kernel( np.asarray(teaching_embeddings), np.asarray(teaching_embeddings))
        for i in range(len(teaching_embeddings)):
            # get all similarities
            similarities_embeds = similarities_embeds_all[i]
            opt_defer_ex = opt_defer_teaching[i]
            opt_gamma = 1
            sorted_sim = sorted([(similarities_embeds[k], opt_defer_teaching[k]) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])
            indicess = list(range(1, len(opt_defer_teaching)))
            indicess.reverse()
            for k in indicess:
                if sorted_sim[k][1] == opt_defer_ex and sorted_sim[k- 1][1] != opt_defer_ex:
                    opt_gamma = sorted_sim[k][0]
                    break
            optimal_gammas.append(opt_gamma)
            pbar.update(1)
    return optimal_gammas

The script below helps get embeddings on teaching and testing set 

In [ ]:

model_classifier.eval()
# now only for rad_1
ai_teaching_preds = []
ai_teaching_conf = []
teaching_target = []
hum_teaching_preds = []
teaching_embeddings = []
print("getting embeddings on teaching set")

with torch.no_grad():
    for j, (input, target) in enumerate(train_val_loader):
        target_np = target.to(device).detach().cpu().numpy()
        input = input.to(device)
        outputs_class = model_classifier(input)
        outputs_conf = outputs_class.data.detach().cpu().numpy()
        _, predicted = torch.max(outputs_class, 1)
        predicted = predicted.detach().cpu().numpy()
        embeddings = model_classifier.get_repr(input)
        embeddings = embeddings.data.detach().cpu().numpy()
        batch_size = outputs_class.size()[0]  # batch_size
        exp_prediction = expert.predict(input, target)
        for i in range(0, batch_size):
            r = (max(outputs_conf[i]) <=0.8 )
            ai_teaching_preds.append(predicted[i])
            ai_teaching_conf.append(max(outputs_class[i]))
            teaching_target.append(target_np[i])
            hum_teaching_preds.append(exp_prediction[i])
            teaching_embeddings.append(embeddings[i])

teaching_embeddings = np.array(teaching_embeddings)







In [ ]:
# get kernel matrix
similarities_embeds_all = rbf_kernel(np.asarray(teaching_embeddings), np.asarray(teaching_embeddings))
sorted_sims = []
print("started")
for i in range(len(similarities_embeds_all)):
    sorted_sim = sorted([(similarities_embeds_all[i][k], k) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])
    sorted_sims.append(np.asarray(sorted_sim))

# Algorithms

## Ours

In [ ]:
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool

indicess = list(range(1, len(teaching_embeddings) -1 ))
indicess.reverse()
def get_improvement_defer_greedy(current_defer_preds, opt_defer_preds, xs, seen_indices):

    error_improvements = []
    error_at_i = 0
    found_gammas = []
    for i in range(len(opt_defer_preds)):
        coin = random.random() # random number between [0,1]

        similarities_embeds = similarities_embeds_all[i]
        sorted_sim = sorted_sims[i] #sorted([(similarities_embeds[k], k) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])

        max_improve = -1000
        gamma_value = optimal_gammas[i]
        current_improve = 0
        so_far = 0
        for j in indicess:
            if i in seen_indices:
                continue

            so_far += 1
            idx = int(sorted_sim[j][1])
            f1_hum = hum_teaching_preds_b[idx]
            f1_ai = ai_teaching_preds_b[idx]
            if opt_defer_preds[i] == 1:
                if current_defer_preds[idx] == 0:
                    current_improve += f1_ai - f1_hum
            else:
                if current_defer_preds[idx] == 1:
                    current_improve += f1_hum - f1_ai

            if current_improve >= max_improve:
                max_improve = current_improve 
                gamma_value = min(optimal_gammas[i], sorted_sim[j][0] )
            
        error_improvements.append(max_improve)
        found_gammas.append(gamma_value)
    return error_improvements, found_gammas


In [ ]:
def teach_ours_doublegreedy():
    human_learner = HumanLearner(None)

    errors = []
    data_sizes  = []
    indices_used = []
    points_chosen = []
    for itt in range(MAX_SIZE):
        print(f'New size {itt}')
        best_index = -1
        # predict with current human learner
        if itt == 0:
            preds_teach = priorhum_teaching_preds
        else:
            preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
        error_improvements, best_gammas = get_improvement_defer_greedy(preds_teach, opt_defer_teaching,  teaching_embeddings, indices_used)
        print(f'got improvements with max {max(error_improvements)}')
        best_index = np.argmax(error_improvements)
        indices_used.append(best_index) # add found element to set used
        ex_embed = teaching_embeddings[best_index]
        ex_label = opt_defer_teaching[best_index]
        gamma = best_gammas[best_index] # + (np.random.rand(1)[0])*(1-optimal_gammas[best_index])-(1-optimal_gammas[best_index])/2 # random choice
        human_learner.add_to_teaching([ex_embed, ex_label, gamma])

        if False and itt % PLOT_INTERVAL == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metricsc, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors.append(metricsc)   
            print("##############################")

        if   itt % PLOT_INTERVAL == 0:
            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(testing_embeddings, priorhum_testing_preds)
            _, metricsc, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors.append(metricsc['accuracy'])   
            print("##############################")
    return errors, indices_used
#errors_doublegreedy, indices_used_doublegreedy = teach_ours_doublegreedy()

In [ ]:
def get_improvement_defer(current_defer_preds, opt_defer_preds, gammas, xs, coin_prob = 0.1):
    error_improvements = []
    #similarities_embeds_all = rbf_kernel(np.asarray(xs), np.asarray(xs))
    error_at_i = 0
    for i in range(len(gammas)):
        coin = random.random() # random number between [0,1]
        error_at_i = 0
        similarities_embeds = similarities_embeds_all[i]
        for j in range(len(similarities_embeds)):
            if similarities_embeds[j] >= gammas[i]:
                f1_hum = hum_teaching_preds_b[j]
                f1_ai = ai_teaching_preds_b[j]
                if opt_defer_preds[i] == 1:
                    if current_defer_preds[j] == 0:
                        error_at_i += f1_ai - f1_hum
                else:
                    if current_defer_preds[j] == 1:
                        error_at_i += f1_hum - f1_ai
        error_improvements.append(error_at_i)

        # get the ball for x
        # in this ball how many does the current defer not match the optimal
    return error_improvements



def teach_ours(greedy_gamma = False):
    human_learner = HumanLearner(None)
    errors = []
    data_sizes  = []
    indices_used = []
    points_chosen = []
    for itt in range(MAX_SIZE):
        print(f'New size {itt}')
        best_index = -1
        # predict with current human learner
        if itt == 0:
            preds_teach = priorhum_teaching_preds
        else:
            preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
        error_improvements = get_improvement_defer(preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
        best_index = np.argmax(error_improvements)
        indices_used.append(best_index) # add found element to set used
        ex_embed = teaching_embeddings[best_index]
        ex_label = opt_defer_teaching[best_index]

        if greedy_gamma:
            _, greedy_gamma = get_greedy_gamma(best_index, preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
            gamma = greedy_gamma
            print(f'got improvements with max {_}')
        else:
            gamma = optimal_gammas[best_index]
            print(f'got improvements with max {max(error_improvements)}')

        #gamma = optimal_gammas[best_index] # + (np.random.rand(1)[0])*(1-optimal_gammas[best_index])-(1-optimal_gammas[best_index])/2 # random choice
        human_learner.add_to_teaching([ex_embed, ex_label, gamma])

        if False and itt % 3 == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metrics, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors.append(metrics)   
            print("##############################")

        if   itt % PLOT_INTERVAL == 0:

            plt.imshow(  train_dataset[best_index][0].permute(1, 2, 0)  )
            plt.show()
            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(testing_embeddings, priorhum_testing_preds)
            _, metrics, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors.append(metrics['accuracy'])   
            print("##############################")
    return errors, indices_used
#errors, indices_used = teach_ours(True)

## Medoids

In [ ]:
from sklearn_extra.cluster import KMedoids

def get_greedy_gamma(i, current_defer_preds, opt_defer_preds, gammas, xs):
    similarities_embeds = similarities_embeds_all[i]
    sorted_sim = sorted_sims[i]#sorted([(similarities_embeds[k], k) for k in range(len(teaching_embeddings))], key=lambda tup: tup[0])
    indicess = list(range(1, len(opt_defer_teaching)))
    indicess.reverse()
    max_improve = -1000
    gamma_value = 1
    current_improve = 0
    so_far = 0

    for j in indicess:

        so_far += 1
        idx = int(sorted_sim[j][1])
        #f1_hum = metric_max_over_ground_truths(f1_score, train_answers[idx], [hum_teaching_preds[idx]]) # pass as param plz
        #f1_ai = metric_max_over_ground_truths(f1_score, train_answers[idx], [ai_teaching_preds[idx]])# pass as param plz
        f1_hum = hum_teaching_preds_b[idx]
        f1_ai = ai_teaching_preds_b[idx]
        if opt_defer_preds[i] == 1:
            if current_defer_preds[idx] == 0:
                current_improve += f1_ai - f1_hum#2*(opt_defer_preds[idx]-0.5)#f1_ai - f1_hum
        else:
            if current_defer_preds[idx] == 1:
                current_improve += f1_hum - f1_ai#2*(0.5-opt_defer_preds[idx]) #f1_hum - f1_ai

        if current_improve >= max_improve:
            max_improve = current_improve 
            gamma_value = sorted_sim[j][0]

    return max_improve, gamma_value


def teach_medoids(greedy_gamma = False):
    
    human_learner_medoid = HumanLearner(None)
    errors_medoid = []
    data_sizes  = []
    indices_used_medoid = []
    points_chosen = []
    
    for itt in range(MAX_SIZE):
        indices_used_medoid = []

        if False and itt % 3 == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner_medoid.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metrics, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors_medoid.append(metrics)   
            print("##############################")

        if itt % PLOT_INTERVAL == 0:
            human_learner_medoid = HumanLearner(None)
            kmedoids = KMedoids(n_clusters=itt+1, method='alternate',max_iter=100).fit(teaching_embeddings)
            teaching_indices = kmedoids.medoid_indices_

            print(f'New size {itt}')
            first_time = True
            for teach_ex_idx in teaching_indices:
                best_index = teach_ex_idx
                if greedy_gamma:
                    if itt == 0 or first_time:
                        preds_teach = priorhum_teaching_preds
                        first_time = False
                    else:
                        preds_teach = human_learner_medoid.predict(teaching_embeddings, priorhum_teaching_preds)
                    _, greedy_gamma = get_greedy_gamma(best_index, preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
                    gamma = greedy_gamma
                else:
                    gamma = optimal_gammas[best_index]
                ex_embed = teaching_embeddings[best_index]
                ex_label = opt_defer_teaching[best_index]
                indices_used_medoid.append(best_index)
                 #+ (np.random.rand(1)[0])*(1-optimal_gammas[best_index])-(1-optimal_gammas[best_index])/2 # random choice
                human_learner_medoid.add_to_teaching([ex_embed, ex_label, gamma])

            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner_medoid.predict(testing_embeddings, priorhum_testing_preds)
            _, metrics, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors_medoid.append(metrics['accuracy'])   
            print("##############################")
    return errors_medoid, indices_used_medoid
#errors_medoid, indices_used_medoid = teach_medoids()

## Random

In [ ]:
def teach_random(greedy_gamma = False):
    human_learner_random = HumanLearner(None)
    errors_random = []
    data_sizes  = []
    indices_used_random = random.sample(list(range(len(teaching_embeddings))), MAX_SIZE) # used to take gradient steps
    points_chosen = []
    for itt in range(MAX_SIZE):
        print(f'New size {itt}')
        best_index = indices_used_random[itt]
        ex_embed = teaching_embeddings[best_index]
        ex_label = opt_defer_teaching[best_index]
        if greedy_gamma:
            if itt == 0 or first_time:
                preds_teach = priorhum_teaching_preds
                first_time = False
            else:
                preds_teach = human_learner_random.predict(teaching_embeddings, priorhum_teaching_preds)
            _, greedy_gamma = get_greedy_gamma(best_index, preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
            gamma = greedy_gamma
        else:
            gamma = optimal_gammas[best_index]
        human_learner_random.add_to_teaching([ex_embed, ex_label, gamma])


        if False and itt % 3 == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner_random.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metrics, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors.append(metrics)   
            print("##############################")

        if   itt % PLOT_INTERVAL == 0:
            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner_random.predict(testing_embeddings, priorhum_testing_preds)
            _, metrics_c, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors_random.append(metrics_c['accuracy'])   
            print("##############################")
    return errors_random, indices_used_random
#errors_random, indices_used_random = teach_random()

## LEARN AI

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier, KNeighborsClassifier
def teach_learnai(greedy_gamma = False, knn_value = 5, sub_sampling = 1000):
    human_learner_learnai = HumanLearner(None)
    errors_learnai = []
    data_sizes  = []
    indices_used_learnai = []
    points_chosen = []
    set_xs = [teaching_embeddings[0]]
    set_ys = [ai_teaching_preds_b[0]]
    for itt in range(MAX_SIZE):
        print(f'New size {itt}')
        best_index = 0
        best_value = 0
        neigh = KNeighborsClassifier(n_neighbors = 1, weights='distance')
        random_teach_subset = random.sample(list(range(len(teaching_embeddings))), sub_sampling) 
        random_test_subset = random.sample(list(range(len(teaching_embeddings))), sub_sampling) 

        for j in random_teach_subset:
            if j in indices_used_learnai:
                continue
            x_try = teaching_embeddings[j]
            y_try = ai_teaching_preds_b[j]
            set_xs.append(x_try)
            set_ys.append(y_try)
            np_set_xs = np.asarray(set_xs)
            np_set_ys = np.asarray(set_ys)
            if len(np_set_xs) > knn_value:
                neigh = KNeighborsClassifier(n_neighbors = knn_value, weights='distance')
            else:
                neigh = KNeighborsClassifier(n_neighbors = 1, weights='distance')
            neigh.fit(np_set_xs, np_set_ys)
            acc = neigh.score(np.asarray([teaching_embeddings[kk] for kk in random_test_subset]), np.asarray([ai_teaching_preds_b[kk] for kk in random_test_subset]))
            if acc >= best_value:
                best_value = acc
                best_index = j
            set_xs = set_xs[:-1]
            set_ys = set_ys[:-1]
        print(best_value)
        indices_used_learnai.append(best_index)
        ex_embed = teaching_embeddings[best_index]
        ex_label = opt_defer_teaching[best_index]
        
        if greedy_gamma:
            if itt == 0 :
                preds_teach = priorhum_teaching_preds
                first_time = False
            else:
                preds_teach = human_learner_learnai.predict(teaching_embeddings, priorhum_teaching_preds)
                
            _, greedy_gamma = get_greedy_gamma(best_index, preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
            gamma = greedy_gamma
        else:
            gamma = optimal_gammas[best_index]
        human_learner_learnai.add_to_teaching([ex_embed, ex_label, gamma])



        if False and itt % 3 == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner_learnai.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metrics, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors.append(metrics)   
            print("##############################")

        if   itt % PLOT_INTERVAL == 0:
            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner_learnai.predict(testing_embeddings, priorhum_testing_preds)
            _, metrics_c, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors_learnai.append(metrics_c['accuracy'])   
            print("##############################")

    return errors_learnai, indices_used_learnai
#errors_learnai, indices_used_learnai = teach_learnai()

## LIME

In [ ]:
def get_improvement_lime(features_covered, xs, indices_used_lime):
    error_improvements = []
    for i in range(len(xs)):
        error_at_i = 0
        if i in indices_used_lime:
            error_at_i = -10000
        for feat_id, feat_val in individual_feature_importance[i].items():
            if features_covered[feat_id] == 0:
                error_at_i +=  global_feature_importance[feat_id]
                    
        error_improvements.append(error_at_i)
        # get the ball for x
        # in this ball how many does the current defer not match the optimal
    
    best_index = np.argmax(error_improvements)
    for feat_id, feat_val in individual_feature_importance[best_index].items():
        if features_covered[feat_id] == 0:
            features_covered[feat_id] = 1
    return error_improvements, features_covered

In [ ]:
def teach_lime(greedy_gamma = False):
    human_learner = HumanLearner(None)
    errors_lime = []
    data_sizes  = []
    indices_used_lime = {}
    points_chosen = []
    features_covered = {}
    for i in range(DATA_DIM):
        features_covered[i] = 0

    for itt in range(MAX_SIZE):
        print(f'New size {itt}')
        best_index = -1
        # predict with current human learner

        error_improvements, features_covered = get_improvement_lime(features_covered, teaching_embeddings[:cutoff_size], indices_used_lime)
        print(f'got improvements with max {max(error_improvements)}')
        best_index = np.argmax(error_improvements)
        indices_used_lime[best_index] =1 # add found element to set used
        ex_embed = teaching_embeddings[best_index]
        ex_label = opt_defer_teaching[best_index]

        if greedy_gamma:
            if itt == 0 :
                preds_teach = priorhum_teaching_preds
            else:
                preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
                
            _, greedy_gamma = get_greedy_gamma(best_index, preds_teach, opt_defer_teaching, optimal_gammas, teaching_embeddings)
            gamma = greedy_gamma
        else:
            gamma = optimal_gammas[best_index]

        #gamma = optimal_gammas[best_index]  #+ (np.random.rand(1)[0])*2*(1-optimal_gammas[best_index])-(1-optimal_gammas[best_index]) # random choice
        human_learner.add_to_teaching([ex_embed, ex_label, gamma])



        if False and itt % 3 == 0:
            print("####### train eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(teaching_embeddings, priorhum_teaching_preds)
            _, metrics, __, ___ = compute_metrics(hum_teaching_preds, ai_teaching_preds, preds_teach, teaching_target, True)
            #errors.append(metrics)   
            print("##############################")

        if   itt % PLOT_INTERVAL == 0:
            print("####### val eval " +str(itt)+ " ###########")
            preds_teach = human_learner.predict(testing_embeddings, priorhum_testing_preds)
            _, metrics_c, __, ___ = compute_metrics(hum_testing_preds, ai_testing_preds, preds_teach, testing_target, True)
            errors_lime.append(metrics_c['accuracy'])   
            print("##############################")

    return errors_lime, indices_used_lime
#errors_lime, indices_used_lime = teach_lime()

# Experimental setup for expert deferral

In [ ]:
expert_k = 6 # expert classes able to classify
expert = synth_expert(expert_k, 10)
MAX_SIZE = 11
PLOT_INTERVAL = 4
greedy_gamma = True
MAX_TRIALS = 1


In [ ]:
# replace with optimization over epsilon
print("Optimizing over prior rejector epsilon paramtetr")
max_eps = 0
max_value = 0
for eps_iter in range(50,100,2):
    value_eps = metrics_print_baseline(model_classifier, expert.predict, 10, test_all_loader, eps_iter/100)['system accuracy'] # test_all_loader train_val_loader
    if value_eps >= max_value:
        print(max_value)
        max_value = value_eps
        max_eps = eps_iter/100
print(f' on test with epsilon {max_eps}')
print( metrics_print_baseline(model_classifier, expert.predict, 10, test_all_loader, max_eps))

In [ ]:
scores_ours = []
scores_medoid = []
scores_random = []
scores_aibaseline = []
scores_oracle = []
scores_lime = []
scores_human = []
for trial in range(MAX_TRIALS):
    # Get Human Predictions
    print(f' \n \n trial {trial}  \n \n')

    model_classifier.eval()
    # now only for rad_1
    ai_teaching_preds = []
    ai_teaching_conf = []
    teaching_target = []
    hum_teaching_preds = []
    teaching_embeddings = []
    print("getting predictions on teaching set")

    priorhum_teaching_preds = []

    with torch.no_grad():
        for j, (input, target) in enumerate(train_val_loader):
            target_np = target.to(device).detach().cpu().numpy()
            input = input.to(device)
            outputs_class = model_classifier(input)
            outputs_conf = outputs_class.data.detach().cpu().numpy()
            _, predicted = torch.max(outputs_class, 1)
            predicted = predicted.detach().cpu().numpy()
            embeddings = model_classifier.get_repr(input)
            embeddings = embeddings.data.detach().cpu().numpy()
            batch_size = outputs_class.size()[0]  # batch_size
            exp_prediction = expert.predict(input, target)
            for i in range(0, batch_size):
                r = (max(outputs_conf[i]) >=0.5 )
                priorhum_teaching_preds.append(r)
                ai_teaching_preds.append(predicted[i])
                ai_teaching_conf.append(max(outputs_class[i]))
                teaching_target.append(target_np[i])
                hum_teaching_preds.append(exp_prediction[i])
                teaching_embeddings.append(embeddings[i])

    teaching_embeddings = np.array(teaching_embeddings)


    model_classifier.eval()
    # now only for rad_1
    ai_testing_preds = []
    ai_testing_conf = []
    testing_target = []
    hum_testing_preds = []
    testing_embeddings = []
    print("getting predictions on testing set")
    priorhum_testing_preds = []

    with torch.no_grad():
        for j, (input, target) in enumerate(test_all_loader):
            target_np = target.to(device).detach().cpu().numpy()
            input = input.to(device)
            outputs_class = model_classifier(input)
            outputs_conf = outputs_class.data.detach().cpu().numpy()
            _, predicted = torch.max(outputs_class, 1)
            predicted = predicted.detach().cpu().numpy()
            embeddings = model_classifier.get_repr(input)
            embeddings = embeddings.data.detach().cpu().numpy()
            batch_size = outputs_class.size()[0]  # batch_size
            exp_prediction = expert.predict(input, target)
            for i in range(0, batch_size):
                r = (max(outputs_conf[i]) >=0.5 )
                priorhum_testing_preds.append(r)
                ai_testing_preds.append(predicted[i])
                ai_testing_conf.append(max(outputs_class[i]))
                testing_target.append(target_np[i])
                hum_testing_preds.append(exp_prediction[i])
                testing_embeddings.append(embeddings[i])


    testing_embeddings = np.array(testing_embeddings)


    opt_defer_teaching = []
    for i in range(len(hum_teaching_preds)):
        # optimal decision is to defer to AI only if not in the first k classes
        if teaching_target[i] <= expert_k :
            opt_defer_teaching.append(0)
        else:
            opt_defer_teaching.append(1)


        '''
        score_hum = hum_teaching_preds[i]== teaching_target[i]
        score_ai = ai_teaching_preds[i] == teaching_target[i]

        if score_ai > score_hum:
            opt_defer_teaching.append(1)
        elif score_hum >= score_ai:
            opt_defer_teaching.append(0)
        else:
            opt_defer_teaching.append(random.randint(0,1))
        '''

    opt_defer_testing = []
    for i in range(len(hum_testing_preds)):
        if testing_target[i] <= expert_k :
            opt_defer_testing.append(0)
        else:
            opt_defer_testing.append(1)
        '''
        score_hum = hum_testing_preds[i] == testing_target[i]
        score_ai = round(ai_testing_preds[i])== testing_target[i]
        if score_ai > score_hum:
            opt_defer_testing.append(1)
        elif score_hum >= score_ai:
            opt_defer_testing.append(0)
        else:
            opt_defer_testing.append(random.randint(0,1))
        '''



    hum_teaching_preds_b = [ (hum_teaching_preds[i] == teaching_target[i]) * 1.0 for i in range(len(teaching_target))] 
    ai_teaching_preds_b = [(ai_teaching_preds[i] == teaching_target[i]) * 1.0  for i in range(len(teaching_target))] 



    ai_linear = LogisticRegression(random_state=0).fit(teaching_embeddings, ai_teaching_preds)
    ai_linear.score(teaching_embeddings, ai_teaching_preds)

    
    
    prior_score = compute_metrics(hum_testing_preds, ai_testing_preds, priorhum_testing_preds, testing_target, True)
    optimal_score = compute_metrics(hum_testing_preds, ai_testing_preds, opt_defer_testing, testing_target, True)
    prior_score = prior_score[1]['accuracy']

    human_score =  compute_metrics(hum_testing_preds, ai_testing_preds, [0]*len(ai_testing_preds), testing_target, False)
    ai_score =  compute_metrics(hum_testing_preds, ai_testing_preds, [1]*len(ai_testing_preds), testing_target, False)

    scores_human.append(human_score[1]['accuracy'])
    scores_oracle.append(optimal_score[1]['accuracy'])
    indicess = list(range(1, len(teaching_embeddings)))
    indicess.reverse()
    if not greedy_gamma:
        optimal_gammas = get_optimal_gammas()
    else:
        optimal_gammas = [1]*len(teaching_embeddings)
        
    print("running medoid")
    errors_medoid, indices_used_medoid = teach_medoids(greedy_gamma)        
        
    print("running our method")
    if greedy_gamma:
        errors, indices_used = teach_ours_doublegreedy()

    else:
        errors, indices_used = teach_ours(False)

    print("running learnai")
    errors_learnai, indices_used_learnai = teach_learnai(greedy_gamma)

    print("running lime")
    DATA_DIM = teaching_embeddings.shape[1]
    explainer = lime.lime_tabular.LimeTabularExplainer(teaching_embeddings, discretize_continuous= False)
    global_feature_importance = {}
    cutoff_size = min(20000,len(teaching_embeddings))
    for i in range(DATA_DIM):
        global_feature_importance[i] = 0
    individual_feature_importance = []
    with tqdm(total=len(teaching_embeddings)) as pbar:
        for ex in range(cutoff_size):
            exp = explainer.explain_instance(teaching_embeddings[ex], ai_linear.predict_proba,  num_samples = 100)
            feat_weights = exp.local_exp[1]
            dic_feat_weights = {}
            for j in range(len(feat_weights)):
                dic_feat_weights[feat_weights[j][0]] = abs(feat_weights[j][1])
                global_feature_importance[feat_weights[j][0]] += abs(feat_weights[j][1])
            individual_feature_importance.append(dic_feat_weights)
            pbar.update(1)

    for i in range(DATA_DIM):
        global_feature_importance[i] = math.sqrt(global_feature_importance[i])
    errors_lime, indices_used_lime= teach_lime(greedy_gamma)

    
    print("running random")
    errors_random, indices_used_random = teach_random(greedy_gamma)
        
               
    errors.insert(0, prior_score)
    errors_learnai.insert(0, prior_score)
    errors_medoid.insert(0, prior_score)
    errors_random.insert(0, prior_score)
    errors_lime.insert(0, prior_score)
    
    scores_ours.append(errors)
    scores_aibaseline.append(errors_learnai)
    scores_medoid.append(errors_medoid)
    scores_random.append(errors_random)
    scores_lime.append(errors_lime)

In [ ]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
def get_conf_interval(arr):
    alpha_level = 0.4
    err  = st.t.interval(alpha_level, len(arr)-1, loc=np.mean(arr), scale=st.sem(arr))[1]/2  - st.t.interval(alpha_level, len(arr)-1, loc=np.mean(arr), scale=st.sem(arr))[0]/2 
    return err


In [ ]:

teaching_sizes = [PLOT_INTERVAL*i for i in range(0,math.floor(MAX_SIZE/PLOT_INTERVAL))]
actual_max_trials = len(scores_ours) 

avgs_rand = [np.average([ scores_oracle[triall] - scores_ours[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([scores_oracle[triall] - scores_ours[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
plt.errorbar(teaching_sizes,  avgs_rand, yerr=stds_rand, marker = "o",  label=f'DOUBLE-GREEDY (Ours)')

print(avgs_rand)
print(stds_rand)
avgs_rand = [np.average([scores_oracle[triall] - scores_medoid[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([ scores_oracle[triall] -scores_medoid[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
plt.errorbar(teaching_sizes,  avgs_rand, yerr=stds_rand, marker = "s",   label=f'K-Medoids')

print(avgs_rand)
print(stds_rand)
avgs_rand = [np.average([scores_oracle[triall] - scores_random[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([scores_oracle[triall] -scores_random[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#plt.errorbar(list(range(1,len(teaching_sizes)+1)),  avgs_rand, yerr=stds_rand, label=f'random')
plt.errorbar(teaching_sizes,  avgs_rand,yerr=stds_rand, marker = "v",  label=f'Random')
print(avgs_rand)
print(stds_rand)

avgs_rand = [np.average([scores_oracle[triall]- scores_aibaseline[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([scores_oracle[triall] -scores_aibaseline[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
plt.errorbar(teaching_sizes,  avgs_rand,yerr=stds_rand, marker = "*",   label=f'AI-Behavior')


avgs_rand = [np.average([scores_oracle[triall] - scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([scores_oracle[triall] -scores_lime[triall][i] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#plt.errorbar(list(range(1,len(teaching_sizes)+1)),  avgs_rand, yerr=stds_rand, label=f'random')
plt.errorbar(teaching_sizes,  avgs_rand,yerr=stds_rand, marker = "+",  label=f'LIME')
avgs_rand = [np.average([scores_oracle[triall] - scores_human[triall] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
stds_rand = [np.std([scores_oracle[triall] -scores_human[triall] for triall in range(actual_max_trials)]) for i in range(len(teaching_sizes))]
#plt.errorbar(list(range(1,len(teaching_sizes)+1)),  avgs_rand, yerr=stds_rand, label=f'random')
#plt.errorbar(teaching_sizes,  avgs_rand,yerr=stds_rand, marker = "x",  label=f'Human alone')
#plt.errorbar(teaching_sizes[0],  avgs_rand[0],yerr=stds_rand[0], marker = "x",  label=f'Human alone {avgs_rand[0]:.2f} $\pm$ {stds_rand[0]:.2f}')



ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()   
plt.grid()
plt.legend(fontsize='large')
plt.legend()
plt.ylabel('Difference to Oracle Accuracy',  fontsize='x-large')
plt.xlabel('Teaching set size', fontsize='x-large')
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 6
fig_size[1] = 4
plt.savefig("teaching_complexity_cifar10.pdf", dpi = 1000)
plt.show()